# Computer Vision: YOLO for identifying Formula 1 Teams

This notebook contains the essential code for training a YOLOv8 model to detect and classify Formula 1 teams from images and videos.

In [ ]:
# Imports
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from ultralytics import YOLO
import cv2
from torch.optim import Adam
import numpy as np
import random
import shutil
import yaml
from collections import Counter
import time

## Path Configuration

Define the paths to your training images and dataset

In [ ]:
# Path constants - update these to match your file structure
TRAIN_IMAGE = r"f1-dataset/train/images/1-2023-Brazilian-GP-FP1-20_jpg.rf.2f3d25867c8e1661f6070ae5a84c6dd4.jpg"
IMAGE_FOLDER = r"f1-dataset/train/images"
DATA_YAML = r"f1-dataset/data.yaml"
PROJECT_DIR = r"yolo-files/runs"
WEIGHTS_DIR = r"weights"

## Check PyTorch Version and GPU Availability

In [ ]:
# Check PyTorch version and CUDA availability
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

## Training Configuration

Define the parameters for training the model

In [ ]:
# Device and training configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = 10  # Number of F1 teams
LEARNING_RATE = 0.0005
BATCH_SIZE = 8  # Adjust based on GPU memory
FINE_TUNED_BATCH_SIZE = 6
NUM_WORKERS = 8  # Increase if you have more CPU cores
NUM_EPOCHS = 50  # Or more, depending on performance
FINE_TUNED_EPOCHS = 30
AUGMENTATION = True  # Data augmentation flag

## Image Verification

Verify that the training images can be loaded correctly

In [ ]:
# Verify a sample image loads correctly
train_image = cv2.imread(TRAIN_IMAGE)
if train_image is None:
    print("Error loading the image")
else:
    print("Image loaded correctly: ", train_image.shape)

## Check Image Sizes

Ensure all images have consistent dimensions

In [ ]:
# Define the image folder
image_folder = IMAGE_FOLDER
# List to store image sizes
sizes = set()

# Check the size of each image
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    img = cv2.imread(image_path)

    if img is not None:
        img_size = img.shape  # (height, width, channels)
        sizes.add(img_size)
    else:
        print(f"Error loading image: {image_name}")
        
# Check if all images have the same size
if len(sizes) == 1:
    print("All images have the same size.")
else:
    print("There are images with different sizes. The sizes found are:")
    for size in sizes:
        print(size)

## Utility Functions

Functions for model setup, training, evaluation and testing

In [ ]:
# For reproducibility
def set_seed(seed=42):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    print(f"Random seed set to {seed}")

In [ ]:
# Load a YOLO model
def load_model(weights_path=None):
    """Load a YOLO model from weights or use pretrained"""
    if weights_path and os.path.exists(weights_path):
        model = YOLO(weights_path)
        print(f"Loaded model from {weights_path}")
    else:
        model = YOLO("yolov8m.pt")  # Load pretrained model
        print("Loaded pretrained YOLOv8m model")
    return model

In [ ]:
# Update data.yaml with class weights
def update_data_yaml(yaml_path, class_weights=None):
    """Update data.yaml with custom class weights"""
    # Read existing data.yaml
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    # Create backup if none exists
    backup_file = yaml_path + '.backup'
    if not os.path.exists(backup_file):
        shutil.copy(yaml_path, backup_file)
        print(f"Backup created at {backup_file}")
    
    # Get class names
    class_names = data_config.get('names', [])
    
    # Update with class weights if provided
    if class_weights:
        data_config['class_weights'] = class_weights
        
        # Save the updated file
        with open(yaml_path, 'w') as f:
            yaml.dump(data_config, f, default_flow_style=False)
        
        print(f"Updated {yaml_path} with custom class weights:")
        for i, name in enumerate(class_names):
            weight = class_weights.get(i, 1.0)
            print(f"  - {name}: {weight}")
    
    return data_config

## Model Training Function

Core function to train the YOLO model with optimized hyperparameters

In [ ]:
# Train a model with given parameters
def train_model(model, data_yaml_path, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, 
                lr=LEARNING_RATE, project_path=PROJECT_DIR, run_name='train'):
    """Train a YOLO model with specified parameters"""
    
    print(f"\n=== TRAINING CONFIGURATION ===")
    print(f"Epochs: {epochs}")
    print(f"Batch size: {batch_size}")
    print(f"Learning rate: {lr}")
    print(f"Project path: {project_path}")
    print(f"Run name: {run_name}")
    print("===================================\n")
    
    # Check if directories exist, create if not
    os.makedirs(project_path, exist_ok=True)
    
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=640,  # Image size
        optimizer="AdamW",
        lr0=lr,
        lrf=0.01,  # Final LR factor
        cos_lr=True,  # Use cosine learning rate schedule
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3,
        
        # Loss weightings
        box=7.5,  # Box loss weight
        cls=7.0,  # Class loss weight
        dfl=1.5,  # Distribution focal loss weight
        
        # Data augmentation 
        mosaic=0.8,        # Mosaic augmentation
        mixup=0.2,         # Mixup augmentation
        copy_paste=0.1,    # Copy-paste augmentation
        degrees=15.0,      # Rotation augmentation
        translate=0.2,     # Translation augmentation
        scale=0.5,         # Scale augmentation
        shear=1.0,         # Shear augmentation
        fliplr=0.5,        # Horizontal flip probability
        flipud=0.05,       # Vertical flip probability
        hsv_h=0.015,       # Hue augmentation
        hsv_s=0.7,         # Saturation augmentation
        hsv_v=0.5,         # Value/brightness augmentation
        
        # Early stopping
        patience=15,
        
        # Device
        device=DEVICE,
        
        # Pretrained weights
        pretrained=True,
        
        # Save settings
        project=project_path,
        name=run_name,
        save_period=5,      # Save checkpoint every 5 epochs
        
        # Additional options
        verbose=True,       # Print training statistics
        val=True,           # Run validation during training
        plots=True,         # Create training plots
    )
    
    print(f"Training completed for {epochs} epochs")
    return results

## Fine-Tuning Function

For fine-tuning an already trained model for better performance on specific classes

In [ ]:
# Fine-tune a model
def fine_tune_model(model, data_yaml_path, epochs=FINE_TUNED_EPOCHS, batch_size=FINE_TUNED_BATCH_SIZE,
                    lr=LEARNING_RATE/3, project_path=PROJECT_DIR, run_name='fine_tune'):
    """Fine-tune a YOLO model with lower learning rate"""
    
    print(f"\n=== FINE-TUNING CONFIGURATION ===")
    print(f"Epochs: {epochs}")
    print(f"Batch size: {batch_size}")
    print(f"Learning rate: {lr} (reduced for fine-tuning)")
    print(f"Project path: {project_path}")
    print(f"Run name: {run_name}")
    print("===================================\n")
    
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=800,            # Larger image size for fine details
        optimizer="AdamW",
        lr0=lr,               # Lower learning rate for fine-tuning
        lrf=0.000001,         # Very low final learning rate
        cos_lr=True,
        momentum=0.937,
        weight_decay=0.001,   # Increased regularization
        warmup_epochs=3,
        
        # Loss weightings - adjusted for fine-tuning
        box=6.0,
        cls=9.0,              # Higher class weight for better classification
        dfl=1.5,
        
        # Less aggressive augmentation for fine-tuning
        mosaic=0.7,
        mixup=0.15,
        copy_paste=0.1,
        degrees=10.0,         # Less rotation
        translate=0.1,
        scale=0.4,
        shear=0.5,            # Less shear
        fliplr=0.5,
        flipud=0.01,          # Minimal vertical flipping
        hsv_h=0.01,           # Less color distortion
        hsv_s=0.5,
        hsv_v=0.3,
        
        # Early stopping - more patience
        patience=20,
        
        # Device
        device=DEVICE,
        
        # Pretrained weights (use the current model)
        pretrained=True,
        
        # Save settings
        project=project_path,
        name=run_name,
        save_period=2,        # Save more frequently
        
        # Additional options
        verbose=True,
        val=True,
        plots=True,
    )
    
    print(f"Fine-tuning completed for {epochs} epochs")
    return results

## Model Evaluation and Saving Functions

In [ ]:
# Evaluate a model
def evaluate_model(model):
    """Evaluate a YOLO model and print metrics"""
    print("\nEvaluating model...")
    metrics = model.val()
    
    print(f"\nEvaluation results:")
    print(f"mAP50: {metrics.box.map50:.4f}")
    print(f"mAP50-95: {metrics.box.map:.4f}")
    print(f"Precision: {metrics.box.precision:.4f}")
    print(f"Recall: {metrics.box.recall:.4f}")
    
    return metrics

In [ ]:
# Save a model
def save_model(model, path):
    """Save a YOLO model to specified path"""
    # Ensure directory exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    model.save(path)
    print(f"Model saved to {path}")

In [ ]:
# Test a model on images
def test_model(model, test_images_path, save_results=True):
    """Test a YOLO model on test images"""
    print(f"Testing model on images in {test_images_path}...")
    
    results = model.predict(
        source=test_images_path,
        conf=0.25,           # Confidence threshold
        iou=0.45,            # IoU threshold
        max_det=20,          # Maximum detections per image
        save=save_results    # Save results to disk
    )
    
    print(f"Testing completed")
    return results

## Example workflow: Class Weight Balancing

Function to update class weights based on model performance

In [ ]:
# Example: Update class weights based on validation results
def balance_class_weights(data_yaml_path, class_performance):
    """Update class weights based on model performance
    
    Args:
        data_yaml_path: Path to data.yaml file
        class_performance: Dictionary with class names as keys and mAP values as values
    """
    # Read existing data.yaml
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    # Get class names
    class_names = data_config.get('names', [])
    
    # Create class weights - higher weight for lower performing classes
    class_weights = {}
    for i, name in enumerate(class_names):
        # Default weight is 1.0
        weight = 1.0
        
        # If we have performance data for this class
        if name in class_performance:
            map_value = class_performance[name]
            
            # Inverse relationship: lower mAP gets higher weight
            if map_value < 0.5:
                weight = 2.0  # Very poor performance, high weight
            elif map_value < 0.7:
                weight = 1.5  # Medium performance, medium weight
            elif map_value < 0.9:
                weight = 1.0  # Good performance, normal weight
            else:
                weight = 0.8  # Excellent performance, slightly lower weight
        
        class_weights[i] = weight
    
    # Update data.yaml with new weights
    return update_data_yaml(data_yaml_path, class_weights)

## Individual Steps

If you prefer to run steps individually, here are examples

In [ ]:
# Example: Load and train a model
# model = load_model()
# train_results = train_model(model, DATA_YAML, epochs=10, run_name='quick_training')

In [ ]:
# Example: Evaluate a model
# metrics = evaluate_model(model)

In [ ]:
# Example: Fine-tune a model for specific classes
# Custom class weights for problematic classes
# class_weights = {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.5, 5: 2.0, 6: 1.0, 7: 1.2, 8: 1.0, 9: 1.0}
# update_data_yaml(DATA_YAML, class_weights)
# fine_tune_results = fine_tune_model(model, DATA_YAML, epochs=15, run_name='class_specific_tuning')

In [ ]:
# Example: Test a saved model
# test_model = load_model('weights/f1_detection_fine_tuned.pt')
# test_results = test_model(test_model, 'f1-dataset/test/images')